<a href="https://colab.research.google.com/github/FateenAnam/Big-Data-Final-Project/blob/main/PlotlyVisualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade plotly
!pip install --upgrade geopandas
!pip install --upgrade pyshp
!pip install --upgrade shapely
!pip install censusdata 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 37.8 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.13.1
    Uninstalling plotly-5.13.1:
      Successfully uninstalled plotly-5.13.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 89.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 58.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 2.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, http

In [8]:
!pip install plotly-geo
#import plotly-geo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import plotly.offline as ofl
import plotly.graph_objs as go
import numpy as np


x = np.arange(1,31)
y = [a**2 for a in x]

trace = go.Scatter(x=x, y=y, marker={'color': 'red', 'size': 10}, 
                    mode="markers",  name='1st Trace')

In [10]:
data=[trace]
layout=go.Layout(title="First Plot", xaxis={'title':'x1'}, yaxis={'title':'x2'})

In [11]:
figure=go.Figure(data=data,layout=layout)
figure.show(renderer="colab")

In [12]:
import pandas as pd
import censusdata
from tabulate import tabulate

In [ ]:
# Based on https://towardsdatascience.com/mapping-us-census-data-with-python-607df3de4b9c

In [ ]:
#FIPS codes are five-digit numbers; for Tennessee the codes start with 47 and are completed with the three-digit county code.
# The FIPS code for each county in the table links to census data for that county.

In [ ]:
#MEANS OF TRANSPORTATION TO WORK
#https://api.census.gov/data/2017/acs/acs5/groups/B08301.html

In [13]:
df = censusdata.download('acs5', 
           2015,
           censusdata.censusgeo([('state', '47'), ('county', '*')]),
           ['B08301_001E', 'B08301_010E']) #Estimate!!Total, #Estimate!!Total!!Public transportation (excluding taxicab)

In [14]:
column_names = ['total_transpo', 'total_public_transpo']
df.columns = column_names

In [15]:
df['percent_public_transpo'] = df.apply(
   lambda row: row['total_public_transpo']/row['total_transpo'], 
   axis = 1)

In [16]:
new_indices = []
county_names = []
for index in df.index.tolist():
        new_index = index.geo[0][1] + index.geo[1][1]
        new_indices.append(new_index)
        county_name = index.name.split(',')[0].split(' ')[0].strip()
        county_names.append(county_name)
df.index = new_indices
df['county_name'] = county_names

In [18]:
from urllib.request import urlopen
import json
import geopandas as gpd

with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = gpd.read_file(response, driver='GeoJSON')
tn=counties[counties.STATE=='47']


In [19]:
tn=tn.merge(df, left_on='NAME',right_on='county_name')

In [20]:
tn.percent_public_transpo.max()

0.02306804385141237

In [21]:
tn=tn.set_index('county_name')

In [22]:
import plotly.express as px
fig = px.choropleth(tn, geojson=tn.geometry, locations=tn.index, color='percent_public_transpo',
                            color_continuous_scale="Viridis",
                            range_color=(0, tn.percent_public_transpo.max()),
                            scope="usa",                            
                            labels={'percent_public_transpo':'Public Transportation Use','count_name':'County Name'})
fig.update_geos(fitbounds="locations", visible=True)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show(renderer="colab")